<a href="https://colab.research.google.com/github/CAP4770-group9-project/final-project/blob/notebook%2Fkmeans/k-means-clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>